In [ ]:
# Guarded Setup
DRY_RUN = True
from notebooks._utils.common import *
CLI_OK = shell_available('forensic-cli')
LAB_ID = '10_case_management_and_coc'
LAB_ROOT = lab_root(LAB_ID)
print(f'CLI available: {CLI_OK}')
print(f'Artifacts root: {LAB_ROOT}')


# Lab 10 · Case Management and Chain of Custody

This lab demonstrates how to initialise a forensic case with the SDK,
capture a small artefact, and verify the resulting chain-of-custody entries.


## Objectives
- Create or load a deterministic demonstration case.
- Record a notebook-generated artefact (≤200 bytes) with a SHA-256 hash.
- Review chain-of-custody events as a pandas table.


## Create or Load the Demonstration Case (SDK)
The SDK owns the primary workflow. We use a fixed case identifier so that
re-running the notebook simply reuses the same case in the lab workspace.


In [ ]:
from pathlib import Path
import pandas as pd
from forensic.core.framework import ForensicFramework
from forensic.utils.hashing import compute_hash

WORKSPACE = LAB_ROOT / 'workspace'
CASE_ID = 'CASE_LAB10_DEMO'
framework = ForensicFramework(workspace=WORKSPACE)

try:
    case = framework.load_case(CASE_ID)
    CASE_CREATED = False
except ValueError:
    case = framework.create_case(
        name='Lab 10 Demonstration Case',
        description='Notebook-managed artefact registration example.',
        investigator='Notebook Analyst',
        case_id=CASE_ID,
    )
    CASE_CREATED = True

case_snapshot = {
    'case_id': case.case_id,
    'name': case.name,
    'investigator': case.investigator,
    'workspace': str(framework.workspace),
    'case_dir': str(case.case_dir),
    'created_this_run': CASE_CREATED,
}
json_dump_sorted(case_snapshot, LAB_ROOT / 'case_snapshot.json')
case_snapshot


### CLI Mirror (optional)
If the CLI is available we ensure it agrees with the SDK-managed state by
running the idempotent `case init` helper in the same workspace.


In [ ]:
if CLI_OK:
    cli_result = run_cli([
        'forensic-cli',
        '--workspace',
        str(WORKSPACE),
        'case',
        'init',
        CASE_ID,
        '--name',
        'Lab 10 Demonstration Case',
        '--description',
        'Notebook-managed artefact registration example.',
        '--investigator',
        'Notebook Analyst',
        '--force',
    ])
    print(cli_result.stdout.strip() or cli_result.stderr or f'Exit code: {cli_result.returncode}')
else:
    print('forensic-cli not detected; SDK workflow already captured the case setup.')


## Register a Notebook Artefact
We create a deterministic evidence note, hash it, and append the metadata to
the case chain-of-custody log. Files remain under `.labs/10_case/` for easy
cleanup between lab runs.


In [ ]:
ARTIFACT_ROOT = LAB_ROOT / 'artifacts'
ARTIFACT_ROOT.mkdir(parents=True, exist_ok=True)
artifact_path = ARTIFACT_ROOT / 'evidence_note.txt'
artifact_content = (
    'Lab 10 evidence note\n'
    'Device: workstation-7\n'
    'Context: Baseline verification for SDK chain-of-custody demo.\n'
)
artifact_path.write_text(artifact_content, encoding='utf-8')
artifact_hash = compute_hash(artifact_path)
framework.append_coc(artifact_path, artifact_hash)
existing_events = framework.coc.get_case_chain(case.case_id)
if not any(event.get('description') == 'Notebook registered evidence note' for event in existing_events):
    framework.coc.log_event(
        event_type='EVIDENCE_REGISTERED',
        actor='Notebook Analyst',
        action='artifact noted',
        description='Notebook registered evidence note',
        case_id=case.case_id,
        metadata={'artifact_path': str(artifact_path)},
        integrity_hash=artifact_hash,
    )
artifact_record = {
    'path': str(artifact_path),
    'sha256': artifact_hash,
}
json_dump_sorted(artifact_record, LAB_ROOT / 'artifact_record.json')
artifact_record


## Inspect Chain-of-Custody Entries
We load the case chain directly from the framework database and render it as
a pandas table for review. The JSONL ledger written by `append_coc` is also
available under the case `meta/` directory.


In [ ]:
events = framework.coc.get_case_chain(case.case_id)
coc_df = pd.DataFrame(events)
coc_df


In [ ]:
coc_ledger = case.case_dir / 'meta' / 'chain_of_custody.jsonl'
print(preview(coc_ledger))


### Checkpoint
The lab is complete once the artefact exists, its hash is recorded, and the
chain-of-custody view contains the registration entry.


In [ ]:
assert artifact_path.exists(), 'Artefact file missing.'
assert artifact_hash, 'Hash computation failed.'
assert not coc_df.empty, 'Chain-of-custody table should not be empty.'
assert 'Notebook registered evidence note' in coc_df['description'].tolist(), 'Expected CoC entry missing.'
